# Uso de bits para aplicar compuertas cuánticas eficientemente

* En lo que sigue, utilizaremos la notación de base 2: 0=[0], [1], 2=[10], 3=[11], etc, y si se usa algún _padding_, hablamos de lo siguiente: 0=[000] (en este ejemplo el padding igual a 3, es decir, siempre veremos todo con tres dígitos, usualmente se utilizará esto con fines didacticos y de claridad).
* Nota importante, independientemente de como indexe Julia, siempre contaremos los qubits de derecha a izquierda y comenzando del 0. Es decir, si tenemos 3 qubits, el de la derecha es el 0 y el de la izquierda el 2.

Como ya vimos en clase, los componentes de los estados de muchos qubits en la base computacional (la base que se construye con los eigenestados de $\sigma_z^{\otimes n}$, por ej $| 0 0 0 0 \rangle$, $| 0 0 0 1 \rangle$ etc.) están convenientemente indexados por números enteros en notación binaria, es decir, el componente tercer componente de un estado de 3 qubits (comenzando a contar desde el $0$) corresponde a 3=[011], lo cual correspondería a la amplitud de probabilidad de medir $|011\rangle$.

En clase también vimos que usualmente tendremos _indices untouched_ e _indices target_, donde los últimos son los indices relevantes donde se aplicaría una unitaria. Para controlar sobre que índices iterar utilizando solo los enteros que indexan a los vectores en Julia, es útil hacer operaciones directamente en los bits de los mismos. En la clase remota nos fuimos paso a paso con el suigente ejemplo:

In [1]:
# Primero definimos algunas funciones auxiliares que nos ayudarán a convertir de un número a una lista de bits y viceversa.
function original_integer(list)
    return parse(Int, join(list); base=2)
end

function base_2(integer; pad= nothing)
    if pad == nothing
        return reverse(digits(integer, base = 2))
    else
        return reverse(digits(integer, base = 2, pad = pad))
    end
end

base_2 (generic function with 1 method)

## Merge Two Integers

La función que queremos contruir es ``merge_two_integers``, la cual recibe 3 enteros ``merge_two_integers(a, b, mask)``, la idea es que los dígitos binarios de ``a`` se colocan en los 1s de ``mask``. 

Por ejemplo: Si la mascara es 2=[010] (mask), target es ${\color{red} 0=[0] (a)}$ y untouched es ${\color{blue} 2=[10] (b)}$, la función nos debe de regresar el entero que surge al "repartir" los indices target sobre los 1s de la mascara, y los untouched sobre los 0s. Es decir, nos debe regresar $[{\color{blue}1}{\color{red} 0}{\color{blue} 0}]=4$. Puse el 0 del target en medio y el 10 en donde están los ceros. Hay que pensar en como hacerlo eficientemente.

A continuación, veremos como serían dos iteraciones (más o menos, hay que hacerlo con cuidado). Observen también lo que hacen los bit-wise operators.

In [52]:
mask = original_integer([0,1,1,0,0])
a = original_integer([1,1])
b = original_integer([1,0,1])
mask, a , b

(12, 3, 5)

In [53]:
original_integer([1,1,1,0,1])

29

In [54]:
mask & 1

0

Utilizar con lo anterior un ``if`` para revisar si toca poner un digito de $a$ o uno de $b$. En este caso toca de $b$, por supuesto.

In [55]:
result = 0 # definimos la variable donde escribiremos lo que nos dará merge_two_integers
bit = b & 1 # Esto nos dice si el último bit de b es 1 o 0
result = result | bit # Esto nos hace que el último bit de result sea igual al último bit de b

1

In [56]:
base_2(result, pad = 5)

5-element Vector{Int64}:
 0
 0
 0
 0
 1

In [57]:
base_2(mask, pad = 5) # Teniendo en mente que esta es la mascara, hay que pasarnos al siguiente bit, eliminando el último bit de mask

5-element Vector{Int64}:
 0
 1
 1
 0
 0

In [58]:
new_mask = mask >> 1
base_2(new_mask, pad = 4) # A sabiendas que eliminé un bit, uso pad = 4, recuerden que el pad es solo para visualiar, el número subyacente no cambia, es para fines ilustrativos

4-element Vector{Int64}:
 0
 1
 1
 0

Antes de proseguir, observen también que ya se utilizó el último dígito de b, por lo tanto hay que quitarlo.

In [59]:
new_b = b >> 1
base_2(new_b, pad = 2) # A sabiendas que eliminé un bit, uso pad = 2, recuerden que el pad es solo para visualiar, el número subyacente no cambia, es para fines ilustrativos

2-element Vector{Int64}:
 1
 0

In [60]:
new_mask & 1 # Vemos que el último bit de la máscara es también cero, por lo tanto de nuevo hay que colocar el último bit de new_b en result

0

In [63]:
bit = new_b & 1 # El segundo bit de b es cero.

0

__Nota importante:__ En lo siguiente, hay que comparar el segundo digito de result con bit, o equivalentemente, recorrer bit por uno y comparar con result. Es decir: result en la primera iteración es 0 = [00001], el 1 a la derecha ya fue añadido de b. Ahora hay que añadir el 0 de b:

In [64]:
result = result | bit<<1

1

In [67]:
bit<<1, bit # En este caso no cambia nada si recorremos bit, ya que igual es cero, pero será relevante cuando el bit a agregar a result sea 1, por ejemplo:

(0, 0)

In [69]:
1<<2 # Aquí recorremos el primer dígito de $1$ dos veces: es decir, mandamos [001] a [100]

4

El siguiente dígito a añadir es el de $a$:

In [73]:
new_mask = mask>>2 # primero recorremos la mascara dos veces
base_2(new_mask, pad = 3)

3-element Vector{Int64}:
 0
 1
 1

In [75]:
# Chequemos si toca a o toca b:
new_mask & 1 # Viendo la máscara, claramente toca a

1

In [76]:
bit = a & 1 # Como no hemos usado a, no hay new_a

1

In [83]:
result = result | bit << 2
base_2(result | bit << 2, pad = 5) # Agregamos el bit recorrido dos veces

5-element Vector{Int64}:
 0
 0
 1
 0
 1

In [84]:
new_a = a>>1 # Como ya usamos el primer dígito de a, lo recorremos

1

In [85]:
result

5